In [27]:
pwd = '/home/zjy/project/MetaIM/data'

In [21]:
import networkx as nx 
from torch_geometric.nn import MessagePassing
import torch
from torch_geometric.utils import dense_to_sparse
import torch.nn.functional as F

class SIR(MessagePassing):
    def __init__(self, N, beta, gamma, C ,steps, device):
        super().__init__(aggr="add")
        self.N = N
        self.beta = beta
        self.gamma = gamma
        self.C = C
        self.steps = steps
        self.device = device

    def init_state(self,x0=None):
        if x0 is None:
            x = torch.zeros(self.N,1).to(self.device)
            ind = torch.argsort(self.w.view(-1),descending=True)
            x[ind[:self.C]] = 1    
        else:
            x = x0    
        r = torch.zeros(self.N,1).to(self.device)
        s = 1 - x
        return s, x ,r
    
    def single_step_forward(self, edge_index, s, x, r):
        q = self.propagate(edge_index, x=torch.log(1-self.beta*x))
        q = torch.exp(q)
        s, x, r = s*q, (1-self.gamma)*x + s*(1-q) , r+ self.gamma*x
        return s, x, r

    def forward(self, edge_index, x0=None):
        s, x, r = self.init_state(x0)
        for i in range(self.steps): 
           s, x ,r = self.single_step_forward(edge_index,s,x,r)
        return s, x, r

    def message(self, x_j):
        return x_j


In [28]:
# spreading model parameters


# graph 
# generate a graph
# N = 200
# k = 6
# G = nx.barabasi_albert_graph(N,k)
# A = torch.from_numpy(nx.to_numpy_array(G))
# edge_index,_ = dense_to_sparse(A)
# degree = A.sum(dim=1)

from torch_geometric.datasets import Planetoid
from torch_geometric.utils import degree

dataset = Planetoid(root=pwd+'/cora', name='cora')
data = dataset[0]
edge_index = data.edge_index
# d = degree(edge_index[1])
# N = data.num_nodes
# # degree-greedy baseline. 
# ind = torch.argsort(d,descending=True)
# x0 = torch.zeros(N,1)
# x0[ind[:C]] = 1
# model = SIR(N,beta,gamma,C,steps)
# _,x,r = model(edge_index, x0=x0)
# print('degree-greddy baseline:',float(r.mean()))

In [80]:
N = data.num_nodes
beta = 0.1
gamma = 1.0
steps = 30
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = SIR(N,beta,gamma,50,steps,device).to(device)

In [38]:
individual_infection = torch.zeros(N,N)
for i in range(N):
    x0 = torch.zeros(N,1).to(device)
    x0[i] = 1
    _,x,r = model(edge_index.to(device), x0=x0)
    individual_infection[i] = r.squeeze().cpu()
individual_infection[0]

tensor([1.0000, 0.0012, 0.0127,  ..., 0.0000, 0.0019, 0.0066])

In [39]:
import numpy as np
np.save(pwd+'/for_meta/cora_individual_infection_sir.npy', individual_infection.numpy())

In [44]:
individual_infection[0][individual_infection[0] > 0.1]

tensor([1.0000, 0.1266, 0.1348, 0.1677, 0.1160, 0.1585, 0.1143, 0.1068, 0.1130,
        0.1010, 0.3529, 0.1123, 0.1240, 0.1070, 0.1034, 0.1343, 0.1015, 0.1175,
        0.1462, 0.1177, 0.1450, 0.1026, 0.1027, 0.1141, 0.1021, 0.1017, 0.1160,
        0.1141, 0.2461, 0.1719, 0.1057, 0.1346, 0.2300, 0.1094, 0.1203, 0.7944,
        0.2071, 0.1032, 0.1076, 0.1059, 0.1518, 0.1796, 0.1084, 0.1082, 0.1121,
        0.1160, 0.1160, 0.1198, 0.1855, 0.1183, 0.1022, 0.1183, 0.1148, 0.1055,
        0.1284, 0.1378, 0.1377, 0.1201, 0.1112, 0.1135, 0.1040, 0.1003, 0.1008,
        0.1010, 0.1052, 0.1904, 0.1262, 0.1329, 0.1773, 0.1132, 0.1110])

In [45]:
individual_infection[0].size()

torch.Size([2708])

5%的初始感染节点

In [84]:
seed_rate = 0.05
C = int(N * seed_rate)
C

135

In [88]:
import random

random.seed(0)

# 从范围 0 到 2800 中随机选择 130 个数字
random_numbers = random.sample(range(N), C)

# 打印随机选择的数字
print(random_numbers)


[1577, 1722, 165, 1060, 2094, 1990, 1658, 1242, 1952, 1466, 2389, 894, 2067, 570, 1154, 572, 388, 2532, 1026, 2181, 2465, 601, 1270, 404, 302, 1352, 1933, 2292, 412, 1449, 1778, 1295, 2502, 2623, 837, 2263, 1953, 1813, 2135, 1066, 255, 2247, 57, 382, 1633, 2561, 4, 2506, 2021, 1364, 999, 1332, 257, 782, 2324, 908, 977, 583, 2224, 1834, 373, 329, 1310, 2080, 2004, 446, 1234, 2257, 1192, 511, 2242, 1362, 2213, 832, 2470, 2241, 2406, 1178, 1822, 375, 2442, 1576, 1298, 2357, 991, 1189, 753, 775, 764, 135, 2509, 2689, 1065, 1951, 282, 367, 533, 612, 158, 328, 2214, 1602, 2148, 1128, 2137, 964, 881, 2415, 1717, 2374, 1127, 1845, 2017, 2704, 2626, 1463, 337, 1328, 472, 1992, 2404, 2581, 1373, 779, 995, 66, 1110, 479, 903, 1523, 698, 1745, 254, 599, 896]


In [86]:
sample_num = 500

seeds_infections = torch.zeros(sample_num, 2, N)
for i in range(500):
    random.seed(i)
    random_seeds = random.sample(range(N), C)
    
    x0 = torch.zeros(N,1).to(device)
    x0[random_seeds] = 1
    _,x,r = model(edge_index.to(device), x0=x0)
    
    seeds_infections[i][0] = x0.squeeze().cpu()
    seeds_infections[i][1] = r.squeeze().cpu()
    

In [92]:
seeds_infections[0][1].sum()

tensor(283.4182)

In [93]:
np.save(pwd+'/for_meta/cora_seed_infection_sir.npy', seeds_infections.numpy())